Martins Python Utilities (mpu) is a collection of utility functions and classes with no other dependencies.

In [ ]:
!pip install mpu

     |████████████████████████████████| 69 kB 4.3 MB/s 


In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import mpu
import os
%matplotlib inline

# ***Load Stuff***

os. getcwd() returns the absolute path of the working directory where Python is currently running as a string str . getcwd stands for "get current working directory", and the Unix command pwd stands for "print working directory". 

In [ ]:
def find_routes_with_ten_readings(df, route_numbers, min_num_readings=10, verbose=False):

    routes = []
    
    for number in route_numbers:
        route_df = df[df['route_number'] == number]
        
        if len(route_df) >= min_num_readings:
            routes.append(route_df)
        elif verbose:
            print('Route: ', number, ' only has ', len(route_df), ' readings!')
    
    print('Found', len(routes), 'routes that have', min_num_readings, 'or more readings')
    
    return pd.concat(routes)

In [ ]:
def load_csv_as_df(file_name, sub_directories, column_numbers=None, column_names=None):
    '''
    Load any csv as a pandas dataframe. Provide the filename, the subdirectories, and columns to read(if desired).
    '''
    base_path = os.getcwd()
    full_path = base_path + sub_directories + file_name

    if column_numbers is not None:
        df = pd.read_csv(full_path, usecols=column_numbers)
    else:
        df = pd.read_csv(full_path)

    if column_names is not None:
        df.columns = column_names
    
    route_ids = df['route_number'].unique()
    return find_routes_with_ten_readings(df, route_ids, min_num_readings=10)

In [ ]:
train_df = load_csv_as_df('all-train-to-air-routes-with-cells.csv', '/')
air_df = load_csv_as_df('all-air-to-train-routes-with-cells(1).csv', '/')

Found 17 routes that have 10 or more readings
Found 21 routes that have 10 or more readings


In [ ]:
train_df.head()

,latitude,longitude,occupancy_status,route_end,route_number,route_start,taxi_id,time,cell,row,column
0,22.608000,114.032600,1,False,324340,True,30907,2018-11-25 06:27:58,12-20,12,20
1,22.608601,114.032097,1,False,324340,False,30907,2018-11-25 06:28:28,12-20,12,20
2,22.607599,114.029900,1,False,324340,False,30907,2018-11-25 06:28:58,12-20,12,20
3,22.606300,114.028099,1,False,324340,False,30907,2018-11-25 06:29:29,12-20,12,20
4,22.604000,114.024597,1,False,324340,False,30907,2018-11-25 06:29:59,12-20,12,20


In [ ]:
air_df.head()

,latitude,longitude,occupancy_status,route_end,route_number,route_start,taxi_id,time,cell,row,column
0,22.626467,113.810150,1,False,2199,True,dodBcDpez8w=,2016-06-13 12:03:02,12-16,12,16
1,22.625467,113.809464,1,False,2199,False,dodBcDpez8w=,2016-06-13 12:03:24,12-16,12,16
2,22.625017,113.808647,1,False,2199,False,dodBcDpez8w=,2016-06-13 12:03:31,12-16,12,16
3,22.614933,113.811600,1,False,2199,False,dodBcDpez8w=,2016-06-13 12:04:53,12-16,12,16
4,22.613100,113.812599,1,False,2199,False,dodBcDpez8w=,2016-06-13 12:05:03,12-16,12,16


In [ ]:
def lookup(s):
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    dates = {date: pd.to_datetime(date) for date in s.unique()}
    return s.map(dates)

def calculate_route_durations(df):
    route_durations = {}
    df['time'] = lookup(df['time'])
    route_ids = df['route_number'].unique()

    for route_id in route_ids:
        route_df = df[df['route_number'] == route_id]

        start_row = route_df[route_df['route_start'] == True]
        end_row = route_df[route_df['route_end'] == True]

        has_start_and_end = True
        if len(start_row) == 0:
            print('No start for route: ', route_id)
            has_start_and_end = False

        if len(end_row) == 0:
            print('No end for route: ', route_id)
            has_start_and_end = False

        if has_start_and_end:
            start_time = start_row['time'].iloc[0]
            end_time = end_row['time'].iloc[0]

            if end_time < start_time:
                print('End time earlier than start time for route number ', route_id)
                print()

            route_duration = end_time - start_time
            # print('route_duration ', route_duration)

            duration_in_seconds = route_duration.total_seconds()

            # print('Route ', route_id, ' duration in seconds ', duration_in_seconds)

            route_durations[route_id] = duration_in_seconds

    duration_df = pd.DataFrame(list(route_durations.items()), columns=['route_number', 'duration_in_seconds'])
    return duration_df

In [ ]:
air_duration_df = calculate_route_durations(air_df)
air_duration_df.describe()

,route_number,duration_in_seconds
count,21.000000,21.000000
mean,229127.523810,2558.285714
std,111253.239972,1136.254335
min,2199.000000,704.000000
25%,161621.000000,1922.000000
50%,235842.000000,2175.000000
75%,320657.000000,3000.000000
max,433086.000000,6384.000000


In [ ]:
train_duration_df = calculate_route_durations(train_df)
train_duration_df.describe()

,route_number,duration_in_seconds
count,17.000000,17.000000
mean,171434.529412,3256.235294
std,136513.619453,1827.482816
min,36889.000000,2045.000000
25%,67941.000000,2368.000000
50%,150097.000000,2610.000000
75%,172162.000000,3055.000000
max,445438.000000,8849.000000


Martins Python Utilities (mpu) is a collection of utility functions and classes with no other dependencies.

In [ ]:
# Calculate the distance on gps
def distance_between_gps(gps_one, gps_two):
    # mpu.haversine_distance((lat1, lon1), (lat2, lon2))
    km_distance = mpu.haversine_distance((gps_one[0], gps_one[1]), (gps_two[0], gps_two[1]))

    if km_distance < 0:
        print('got negative distance that\'s weak')
        km_distance *= -1

    return km_distance

In [ ]:
def calculate_route_distances(df):
    route_distances = {}
    df['time'] = lookup(df['time'])
    route_ids = df['route_number'].unique()

    for route_id in route_ids:
        route_df = df[df['route_number'] == route_id]
        route_df.sort_values('time')
        route_df.reset_index(drop=True)

        distance_sum = 0.0
        is_first_row = True
        
        for index, row in route_df.iterrows():
            if not is_first_row:
                last_row = route_df.loc[index - 1]
                last_lat = last_row['latitude']
                last_long = last_row['longitude']
                last_gps = (last_lat, last_long)

                current_lat = row['latitude']
                current_long = row['longitude']
                current_gps = (current_lat, current_long)

                distance_between_rows = distance_between_gps(last_gps, current_gps)
                distance_sum += distance_between_rows
            else:
                is_first_row = False

        route_distances[route_id] = distance_sum

    distance_df = pd.DataFrame(list(route_distances.items()), columns=['route_number', 'distance_in_km'])
    return distance_df

In [ ]:
train_distance_df = calculate_route_distances(train_df)
train_distance_df.describe()


,route_number,distance_in_km
count,17.000000,17.000000
mean,171434.529412,34.721731
std,136513.619453,5.008724
min,36889.000000,27.655585
25%,67941.000000,32.195178
50%,150097.000000,34.183330
75%,172162.000000,35.765072
max,445438.000000,50.283815


In [ ]:
air_distance_df = calculate_route_distances(air_df)
air_distance_df.describe()

,route_number,distance_in_km
count,21.000000,21.000000
mean,229127.523810,40.303364
std,111253.239972,8.374633
min,2199.000000,31.366844
25%,161621.000000,34.980629
50%,235842.000000,38.547900
75%,320657.000000,42.499935
max,433086.000000,68.183396


In [ ]:
train_time_distance_df=pd.merge(train_duration_df,train_distance_df,on="route_number")
#then convert into csv
train_time_distance_df.to_csv('all-train-to-air-routes-distance-time.csv',encoding='utf-8',index=False)
train_time_distance_df.describe()

,route_number,duration_in_seconds,distance_in_km
count,17.000000,17.000000,17.000000
mean,171434.529412,3256.235294,34.721731
std,136513.619453,1827.482816,5.008724
min,36889.000000,2045.000000,27.655585
25%,67941.000000,2368.000000,32.195178
50%,150097.000000,2610.000000,34.183330
75%,172162.000000,3055.000000,35.765072
max,445438.000000,8849.000000,50.283815


In [ ]:
train_time_distance_df = pd.merge(train_duration_df, train_distance_df, on="route_number")
train_time_distance_df.to_csv('all-train-to-air-routes-distance-time.csv', encoding='utf-8', index=False)
train_time_distance_df.describe()

,route_number,duration_in_seconds,distance_in_km
count,17.000000,17.000000,17.000000
mean,171434.529412,3256.235294,34.721731
std,136513.619453,1827.482816,5.008724
min,36889.000000,2045.000000,27.655585
25%,67941.000000,2368.000000,32.195178
50%,150097.000000,2610.000000,34.183330
75%,172162.000000,3055.000000,35.765072
max,445438.000000,8849.000000,50.283815


In [ ]:
air_time_distance_df = pd.merge(air_duration_df, air_distance_df, on="route_number")
air_time_distance_df.to_csv('all-air-to-train-routes-distance-time.csv', encoding='utf-8', index=False)
air_time_distance_df.describe()

,route_number,duration_in_seconds,distance_in_km
count,21.000000,21.000000,21.000000
mean,229127.523810,2558.285714,40.303364
std,111253.239972,1136.254335,8.374633
min,2199.000000,704.000000,31.366844
25%,161621.000000,1922.000000,34.980629
50%,235842.000000,2175.000000,38.547900
75%,320657.000000,3000.000000,42.499935
max,433086.000000,6384.000000,68.183396


In [ ]:
def merge_distance_time_into_route_df(dt_df, df):
    route_dfs = []
    route_ids = df['route_number'].unique()

    for route_id in route_ids:
        route_df = df[df['route_number'] == route_id]
        distance_time_df = dt_df[dt_df['route_number'] == route_id]
        
        distance = distance_time_df['distance_in_km'].iloc[0]
        time = distance_time_df['duration_in_seconds'].iloc[0]
        
        route_df['distance_in_km'] = distance
        route_df['duration_in_seconds'] = time
        
        route_dfs.append(route_df)
    
    return pd.concat(route_dfs)


In [ ]:
def reduce_dataframe_by_col(df, col_name):
    row_dfs = []
    unique_values = df[col_name].unique()

    for val in unique_values:
        val_df = df[df[col_name] == val]

        row_dfs.append(val_df.iloc[[0]])
    
    return pd.concat(row_dfs)

In [ ]:
merged_air = merge_distance_time_into_route_df(air_time_distance_df, air_df)
merged_air_reduced = reduce_dataframe_by_col(merged_air, 'route_number')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [ ]:
merged_train = merge_distance_time_into_route_df(train_time_distance_df, train_df)
merged_train_reduced = reduce_dataframe_by_col(merged_train, 'route_number')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [ ]:
merged_train_reduced.describe()

,latitude,longitude,occupancy_status,route_number,row,column,distance_in_km,duration_in_seconds
count,17.000000,17.000000,17.0,17.000000,17.0,17.0,17.000000,17.000000
mean,22.611076,114.028252,1.0,171434.529412,12.0,20.0,34.721731,3256.235294
std,0.002638,0.002232,0.0,136513.619453,0.0,0.0,5.008724,1827.482816
min,22.605516,114.026253,1.0,36889.000000,12.0,20.0,27.655585,2045.000000
25%,22.610317,114.026871,1.0,67941.000000,12.0,20.0,32.195178,2368.000000
50%,22.612350,114.027550,1.0,150097.000000,12.0,20.0,34.183330,2610.000000
75%,22.613001,114.028030,1.0,172162.000000,12.0,20.0,35.765072,3055.000000
max,22.613508,114.033966,1.0,445438.000000,12.0,20.0,50.283815,8849.000000


In [ ]:
train_fraud_suspect = merged_train_reduced[(merged_train_reduced['distance_in_km'] >= 34.73) & (merged_train_reduced['duration_in_seconds'] >= 3257)]
print(train_fraud_suspect)

       latitude   longitude  ...  distance_in_km  duration_in_seconds
3078  22.611549  114.027649  ...       35.694714               4531.0

[1 rows x 13 columns]


In [ ]:
train_fraud_suspect

,latitude,longitude,occupancy_status,route_end,route_number,route_start,taxi_id,time,cell,row,column,distance_in_km,duration_in_seconds
3078,22.611549,114.027649,1,False,172162,True,gB7yn7HysUY=,2016-06-13 14:38:18,12-20,12,20,35.694714,4531.0


In [ ]:
merged_air_reduced.describe()

,latitude,longitude,occupancy_status,route_number,row,column,distance_in_km,duration_in_seconds
count,21.000000,21.000000,21.0,21.000000,21.0,21.0,21.000000,21.000000
mean,22.625651,113.809754,1.0,229127.523810,12.0,16.0,40.303364,2558.285714
std,0.003064,0.001072,0.0,111253.239972,0.0,0.0,8.374633,1136.254335
min,22.612907,113.806870,1.0,2199.000000,12.0,16.0,31.366844,704.000000
25%,22.626053,113.809769,1.0,161621.000000,12.0,16.0,34.980629,1922.000000
50%,22.626579,113.809937,1.0,235842.000000,12.0,16.0,38.547900,2175.000000
75%,22.626833,113.810066,1.0,320657.000000,12.0,16.0,42.499935,3000.000000
max,22.627016,113.812714,1.0,433086.000000,12.0,16.0,68.183396,6384.000000


In [ ]:
merged_air_reduced[(merged_air_reduced['distance_in_km'] >= 40.31) & (merged_air_reduced['duration_in_seconds'] >= 2559)]

,latitude,longitude,occupancy_status,route_end,route_number,route_start,taxi_id,time,cell,row,column,distance_in_km,duration_in_seconds
1099,22.626698,113.808090,1,False,208721,True,24181,2018-11-25 08:21:16,12-16,12,16,41.068584,3000.0
1310,22.626711,113.809944,1,False,235842,True,25533,2018-11-25 18:58:12,12-16,12,16,42.499935,2642.0
1577,22.627001,113.809769,1,False,293275,True,28870,2018-11-25 03:08:24,12-16,12,16,45.793026,3029.0
1703,22.626499,113.809982,1,False,320657,True,30647,2018-11-25 14:10:23,12-16,12,16,68.183396,6384.0
2686,22.626833,113.809853,1,False,329370,True,31169,2018-11-25 09:16:59,12-16,12,16,50.143033,3891.0
